In [2]:
import pandas as pd 
import numpy as np 
import pandas_datareader as web
import matplotlib.pyplot as plt 
from datetime import date
import yfinance as yf


c:\users\lenovo\appdata\local\programs\python\python38-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
df = yf.download('SBIN.NS',start=date(2017,4,1),end=date(2018,4,1))
df=df.dropna()

[*********************100%***********************]  1 of 1 completed


In [4]:
n=14
delta =df['Close'].diff()
dUp, dDown = delta.copy(), delta.copy()
dUp[dUp<0]=0
dDown[dDown>0]=0
RolUp = dUp.rolling(n).mean()
RolDown = dDown.rolling(n).mean().abs()
df['RSI'] = np.where(RolDown !=0, RolUp/RolDown,1)
df['RSI_slow']= df['RSI'].rolling(window =5).mean()


In [5]:
df = df.assign (Signal=pd.Series(np.zeros(len(df))).values)
df.loc[df['RSI']>df['RSI_slow'],'Signal']=1
df.loc[df['RSI']<df['RSI_slow'],'Signal']=-1
df['Return']=np.log(df['Close']/df['Close'].shift(1))
df['s_return']=df['Signal'].shift(1)*df['Return']
df['Market return']=df['Return'].expanding().sum()
df['Strategy_Return']=df['s_return'].expanding().sum()


In [6]:
plt.plot(df['Market_Return'],color='r',label='Market Returns')
plt.plot(df['Strategy_Return'],color='b',label='Strategy returns')
plt.legend(loc='best')
plt.show()


KeyError: 'Market_Return'

In [ ]:
df = pd.DataFrame(columns = ['Date', 'Company','Money','Stock','Profit','cumilative profit','B/S']) 
k = 0
for i in names:
    company = yf.download(i,start=date(start_year,4,1),end=date(end_year,4,1))
    emas = []
    emal = []
    emas = (company['Close'].ewm(span = window_small).mean())
    emal = (company['Close'].ewm(span = window_large).mean())
    l = len(emal)
    money = 10000
    profit = 0
    stocks = 0
    buy = []
    cum_profit = 0
    sell = []
    buy_price = 0
    print(i)
    for j in range(l):
        if(emas[j-1] <= emal[j-1] and emas[j] > emal[j]):
            if(money==10000):
                money = 0
                stocks = 10000/company['Close'][j]
            else:
                stocks = money/company['Close'][j]
                money = 0 
            df.loc[k] = [company.index[j],i,money,stocks,profit,'-','B']
            k = k + 1
            buy_price = company['Close'][j]
            print(money,stocks,profit)
        if(emas[j-1] >= emal[j-1] and emas[j] < emal[j]):
            if(money == 0):
                money = stocks*company['Close'][j]
                stocks = 0
                profit = money - 10000
                if(money>10000):
                    cum_profit = cum_profit + profit
                    money = 10000
                    df.loc[k] = [company.index[j],i,money,stocks,profit,cum_profit,'S']
                else:
                    df.loc[k] = [company.index[j],i,money,stocks,profit,'-','S']
            k = k + 1
            print(money,stocks,profit)
            profit = 0
    if(stocks!=0):
        money = stocks*company['Close'][-1]
        profit = money - 10000
        if(money>10000):
            cum_profit = cum_profit + profit
            money = 10000
        df.loc[k] = [company.index[-1],i,money,stocks,profit,cum_profit,'S']
    print(cum_profit+money)
    returns.append(cum_profit+money)
    print()
    print()
CAGR = []
sharpe_ratio = []
rfr = 7
for i in returns:
    value = ((i/10000)**(end_year-start_year) - 1)*100
    CAGR.append(value)
df.to_csv("data.csv")